# Setfit few shot learning

In [ ]:
#!pip install setfit

In [2]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer

## Load custom dataset

In [12]:
train_dataset = load_dataset('csv', data_files='data/smabuse_train.csv', split='train')
eval_dataset = load_dataset('csv', data_files='data/smabuse_test.csv')

Using custom data configuration default-917a8592a98c8f43
Found cached dataset csv (/home/gary_hutson/.cache/huggingface/datasets/csv/default-917a8592a98c8f43/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-e5b069aa1d244461
Found cached dataset csv (/home/gary_hutson/.cache/huggingface/datasets/csv/default-e5b069aa1d244461/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

# Push our custom dataset to the hub

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
train_dataset.push_to_hub('socialmedia-abuse_train')
eval_dataset.push_to_hub('socialmedia-abuse_test')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

## Pull dataset from HuggingFace hub

In [7]:
dataset = load_dataset('StatsGary/socialmedia-abuse')
print(dataset)

Using custom data configuration StatsGary--socialmedia-abuse-b759fa5c3af35781
Found cached dataset parquet (/home/gary_hutson/.cache/huggingface/datasets/StatsGary___parquet/StatsGary--socialmedia-abuse-b759fa5c3af35781/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Text', 'Label'],
        num_rows: 16
    })
})


## Simulate few shot learning by sampling 8 examples per class

In [8]:
num_class = 2
train_ds = dataset['train'].shuffle(seed=123).select(range(8 * num_class))

In [9]:
train_ds['Text']

['Ugly and talentless',
 'I think there nose massive on screen',
 'Obesity on the rise',
 'They make a brilliant actor on screen',
 'You look so fat in that picture',
 'Fat and proud',
 "People suffer from body issues and that's okay",
 'How unattractive can one person be?',
 'What a talented and creative person',
 'Yuck - far too skinny',
 'Look at that horrible body',
 'Your appearance makes me sick',
 'You go and rock those curves',
 'They have to be the ugliest person I have ever seen on screen',
 'How attractive ',
 'Peoples appearance does not matter, it is what their character is like that is important']

## Load a model from hub

In [10]:
model = SetFitModel.from_pretrained('sentence-transformers/paraphrase-mpnet-base-v2')

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


## Create Trainer

In [ ]:
trainer = SetFitTrainer(
    model=model, 
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss_class=CosineSimilarityLoss, 
    batch_size=16,
    num_iterations=20,# The number of text pairs to generate for contrastive learning
    column_mapping={"text": "text", "label": "label"}
)

In [ ]:
trainer.train()

## Evaluate the model

In [ ]:
metrics = trainer.evaluate()

In [ ]:
print(metrics)

## Publish model to hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
!huggingface-cli whoami

In [ ]:
!git config --global credential.helper store

## Push the model to the hub

In [ ]:
MODEL_NAME = 'StatsGary/setfit-ft-sentinent-eval'
trainer.push_to_hub(repo_path_or_name=MODEL_NAME)

## Load model from Hub for inference

In [ ]:
model_infer = SetFitModel.from_pretrained(MODEL_NAME)

In [ ]:
preds = model_infer(["i loved the spiderman movie!", "I hate white people"]) 

In [ ]:
print(preds)

## Access via endpoint

Skip the below if you want to perform local inference. 

In [ ]:
import json
import requests as r

ENDPOINT_URL=""# url of your endpoint
HF_TOKEN="" #Your HF token

# payload samples
regular_payload = { "inputs": "Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need"}

# HTTP headers for authorization
headers= {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

# send request
response = r.post(ENDPOINT_URL, headers=headers, json=paramter_payload)
classified = response.json()

print(classified)